In [ ]:
using Pkg
using Revise
Pkg.activate("..") 
Pkg.instantiate()
using JPEC, Plots
gr() 

In [ ]:
using Revise
include("../src/DCON/fourfit.jl")
import .fourfit_make_metric, .MetricData, .fourfit_make_matrix, .MatrixData, .compute_eigenvalues

### Reading Eq

In [ ]:
# 1. Define the input parameters for the equilibrium solver.
#    - eq_filename: The name of the g-file we just created.
#    - eq_type: "efit" for a standard g-file.
#    - jac_type: "boozer" or "hamada" for the output coordinates.
#    - mpsi, mtheta: Resolution of the output grid.
    equil_control = JPEC.Equilibrium.EquilibriumControl(;
    eq_filename="beta_1.00",        # eq_filename
    eq_type="efit",          # eq_type
    jac_type="hamada",        # jac_type
    grid_type="ldp",
    psilow=0.01,             # psilow
    psihigh=0.990)             # psihigh
equil_config = JPEC.Equilibrium.EquilibriumConfig(equil_control,JPEC.Equilibrium.EquilibriumOutput())
# 2. Run the main equilibrium setup function.
#    This will read the file, solve the direct problem, and return the final object.
println("Starting equilibrium reconstruction...")
plasma_eq = JPEC.Equilibrium.setup_equilibrium(equil_config)
#plasma_eq = JPEC.Equilibrium.setup_equilibrium("./DIIID_example/equil.toml")
println("Equilibrium reconstruction complete.")

In [ ]:
fieldnames(typeof(plasma_eq.rzphi))

### CAlc. Metric

In [ ]:
include("../src/DCON/fourfit.jl")
import .fourfit_make_metric, .MetricData, .fourfit_make_matrix, .MatrixData, .compute_eigenvalues

In [ ]:
# Fourfit Metric Calculation Example
println("📚 Fourfit Metric Calculation")
println("="^30)

using .FourfitMetric
metric_result = nothing
try
    # Metric tensor calculation
    metric_result = FourfitMetric.fourfit_make_metric(
        plasma_eq.rzphi,  # 2D geometry spline
        plasma_eq.sq;     # 1D profile spline
        mpsi=128,          # Radial grid points
        mtheta=256,        # Poloidal grid points  
        mband=10,          # Fourier bandwidth
        fft_flag=true,    # Use FFT
        verbose=true      # Detailed output
    )
    
    println("✅ Metric calculation complete!")
    println("   Grid size: $(size(metric_result.fs))")
 
catch e
    println("❌Error: $e")
end

In [ ]:
include("../src/DCON/fourfit.jl")
import .fourfit_make_metric, .MetricData, .fourfit_make_matrix, .MatrixData, .compute_eigenvalues

### Calc. Matric

In [ ]:
using Revise

In [ ]:
include("../src/DCON/fourfit.jl")
import .fourfit_make_metric, .MetricData, .fourfit_make_matrix, .MatrixData, .compute_eigenvalues

In [ ]:
# Fourfit Matrix Calculation - Complete Example
println("🚀 Fourfit Matrix Calculation")
println("="^40)

using .FourfitMetric
matrix_result = nothing
try
    # Step 1: Matrix calculation with correct arguments
    println("2️⃣  Calculating MHD Coefficient Matrix...")
    
    matrix_result = FourfitMetric.fourfit_make_matrix(
        metric_result,       # metric tensor data
        plasma_eq.sq,        # 1D profile spline
        plasma_eq.rzphi,     # 2D geometry spline
        1.367;               # psio (toroidal flux normalization)
        nn=1,                # toroidal mode number
        mlow=-4,             # minimum poloidal mode
        mhigh=4,             # maximum poloidal mode
        power_flag=false,
        feval_flag=false,
        sas_flag=true,
        verbose=true
    )
    
    # Step 2: Analyze results
    println("3️⃣  Analyzing Results:")
    println("   Matrix storage sizes:")
    println("      F-matrix: $(size(matrix_result.fmats))")
    println("      G-matrix: $(size(matrix_result.gmats))")
    println("      K-matrix: $(size(matrix_result.kmats))")
    println("   Mode range: $(matrix_result.mlow) to $(matrix_result.mhigh)")
    println("   Number of modes: $(matrix_result.mpert)")
    
    println("✅ Matrix calculation completed successfully!")
    
catch e
    println("❌ Error occurred: $e")
    println("🔍 Error type: $(typeof(e))")
end

### Redaing imats(original DCON output)

### comparison

In [ ]:
matrix_result.mband

In [ ]:
reshape(matrix_result.cmats[129,:], 9, 9)

In [ ]:
reader.C[end, :, :]

In [ ]:
(reshape(matrix_result.cmats[64, :], 9, 9)./reader.C[64,:,:])

In [ ]:
reshape(matrix_result.cmats[64, :], 9, 9)

In [ ]:
reader.C[64,:,:]

In [ ]:
reader.C[64,:,:]
sq=plasma_eq.sq
ipsi =64
psifac = sq.xs[ipsi + 1]

f_vals, f1_vals = JPEC.SplinesMod.spline_eval(sq, [psifac], 1)
f_vals[1,:][4]
f1_vals[1,:][2]

In [ ]:
using CSV, DataFrames

mutable struct ImatsReaderCSV
    psi       :: Vector{Float64}       # ψ value list
    m_values  :: Vector{Int}           # m value list
    mpsi      :: Int                   # number of ψ values
    mpert     :: Int                   # number of modes
    A         :: Array{ComplexF64,3}   # matrix A (ψ x mpert x mpert)
    B         :: Array{ComplexF64,3}   # matrix B
    C         :: Array{ComplexF64,3}   # matrix C
    D         :: Array{ComplexF64,3}   # matrix D
    E         :: Array{ComplexF64,3}   # matrix E
    H         :: Array{ComplexF64,3}   # matrix H
end

function ImatsReaderCSV(path::AbstractString)
    # CSV 파일을 헤더는 6번째 줄부터 읽고, 공백을 구분자로 사용
    df = CSV.read(path, DataFrame; skipto=6, delim=' ', ignorerepeated=true)
    println("Found columns: ", names(df))
    
    # 실제 헤더에 맞게 컬럼 이름 지정
    # 컬럼: "psi", "m1", "m2", "real(Ai)", "imag(Ai)", "real(Bi)", "imag(Bi)", 
    #      "real(Ci)", "imag(Ci)", "real(Di)", "imag(Di)", "real(Ei)", "imag(Ei)", 
    #      "real(Hi)", "imag(Hi)"
    psi_vals = sort(unique(df[!, "psi"]))
    m_vals = sort(unique(vcat(df[!, "m1"], df[!, "m2"])))
    mpsi = length(psi_vals)
    mpert = length(m_vals)
    shp = (mpsi, mpert, mpert)
    
    # 빈 행렬 생성
    A = zeros(ComplexF64, shp)
    B = zeros(ComplexF64, shp)
    C = zeros(ComplexF64, shp)
    D = zeros(ComplexF64, shp)
    E = zeros(ComplexF64, shp)
    H = zeros(ComplexF64, shp)
    
    # 값 → 인덱스 변환 사전 생성
    psi_idx = Dict(v => i for (i, v) in enumerate(psi_vals))
    m_idx = Dict(v => i for (i, v) in enumerate(m_vals))
    
    # 각 행(row)마다, 해당 (ψ, m1, m2) 인덱스를 찾아 값을 저장
    for row in eachrow(df)
        k = psi_idx[row["psi"]]
        i = m_idx[row["m1"]]
        j = m_idx[row["m2"]]
        A[k, i, j] = ComplexF64(row["real(Ai)"], row["imag(Ai)"])
        B[k, i, j] = ComplexF64(row["real(Bi)"], row["imag(Bi)"])
        C[k, i, j] = ComplexF64(row["real(Ci)"], row["imag(Ci)"])
        D[k, i, j] = ComplexF64(row["real(Di)"], row["imag(Di)"])
        E[k, i, j] = ComplexF64(row["real(Ei)"], row["imag(Ei)"])
        H[k, i, j] = ComplexF64(row["real(Hi)"], row["imag(Hi)"])
    end
    
    return ImatsReaderCSV(psi_vals, m_vals, mpsi, mpert, A, B, C, D, E, H)
end

In [ ]:
reader=ImatsReaderCSV("../src/DCON/validate/imats.out")

In [ ]:
@show (reshape(matrix_result.cmats[120, :], 9, 9)./reader.C[120,:,:])

In [ ]:
@show (reshape(matrix_result.dmat, 9, 9)./reader.D[64,:,:])